In [ ]:
# !wget https://datahack-prod.s3.amazonaws.com/sample_submission/sample_submission_znWiLZ4.csv
# !wget https://datahack-prod.s3.amazonaws.com/test_file/test_hXY9mYw.csv
# !wget https://datahack-prod.s3.amazonaws.com/train_file/train_MpHjUjU.csv

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score
from sklearn.utils import resample

from sklearn.metrics import fbeta_score, make_scorer
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report


pd.options.display.max_columns = 100

In [3]:
raw_train = pd.read_csv("train_MpHjUjU.csv")
raw_test = pd.read_csv("test_hXY9mYw.csv")

In [4]:
raw_train.head(3)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2


In [5]:
raw_test.head(3)

,Emp_ID
0,394
1,173
2,1090


In [6]:
raw_train.shape,raw_test.shape

((19104, 13), (741, 1))

In [7]:
raw_train.isna().sum().div(raw_train.shape[0])

MMM-YY                  0.00000
Emp_ID                  0.00000
Age                     0.00000
Gender                  0.00000
City                    0.00000
Education_Level         0.00000
Salary                  0.00000
Dateofjoining           0.00000
LastWorkingDate         0.91541
Joining Designation     0.00000
Designation             0.00000
Total Business Value    0.00000
Quarterly Rating        0.00000
dtype: float64

In [8]:
raw_train.describe()

,Emp_ID,Age,Salary,Joining Designation,Designation,Total Business Value,Quarterly Rating
count,19104.000000,19104.000000,19104.000000,19104.000000,19104.000000,1.910400e+04,19104.000000
mean,1415.591133,34.650283,65652.025126,1.690536,2.252670,5.716621e+05,2.008899
std,810.705321,6.264471,30914.515344,0.836984,1.026512,1.128312e+06,1.009832
min,1.000000,21.000000,10747.000000,1.000000,1.000000,-6.000000e+06,1.000000
25%,710.000000,30.000000,42383.000000,1.000000,1.000000,0.000000e+00,1.000000
50%,1417.000000,34.000000,60087.000000,1.000000,2.000000,2.500000e+05,2.000000
75%,2137.000000,39.000000,83969.000000,2.000000,3.000000,6.997000e+05,3.000000
max,2788.000000,58.000000,188418.000000,5.000000,5.000000,3.374772e+07,4.000000


In [9]:
raw_train['Emp_ID'].value_counts()

2049    24
1191    24
1655    24
926     24
1623    24
        ..
1448     1
748      1
930      1
962      1
1194     1
Name: Emp_ID, Length: 2381, dtype: int64

In [10]:
raw_train[raw_train['Emp_ID'] == 1623]

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
10852,2016-01-01,1623,35,Male,C5,Master,94437,2012-08-02,NaN,1,3,484610,2
10853,2016-02-01,1623,36,Male,C5,Master,94437,2012-08-02,NaN,1,3,1028340,2
10854,2016-03-01,1623,36,Male,C5,Master,94437,2012-08-02,NaN,1,3,1918950,2
10855,2016-04-01,1623,36,Male,C5,Master,94437,2012-08-02,NaN,1,3,0,3
10856,2016-05-01,1623,36,Male,C5,Master,94437,2012-08-02,NaN,1,3,1250000,3
10857,2016-06-01,1623,36,Male,C5,Master,94437,2012-08-02,NaN,1,3,116000,3
10858,2016-07-01,1623,36,Male,C5,Master,94437,2012-08-02,NaN,1,3,0,2
10859,2016-08-01,1623,36,Male,C5,Master,94437,2012-08-02,NaN,1,3,0,2
10860,2016-09-01,1623,36,Male,C5,Master,94437,2012-08-02,NaN,1,3,1050000,2
10861,2016-10-01,1623,36,Male,C5,Master,94437,2012-08-02,NaN,1,3,2250010,3


In [11]:
common = set(raw_train['Emp_ID'].unique()).intersection(set(raw_test['Emp_ID'].unique()))
len(common),raw_test.shape

(741, (741, 1))

In [12]:
df = raw_train.copy()
df['istest'] = np.where(df['Emp_ID'].isin(raw_test['Emp_ID']),1,0)
df.sample(3)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,istest
6800,2017-08-01,1021,37,Male,C29,College,25106,2017-06-20,NaN,1,1,0,1,0
13559,2016-10-01,2019,39,Female,C16,Bachelor,64781,2016-06-10,NaN,2,2,545280,2,0
2956,2017-06-01,440,40,Male,C16,Bachelor,113125,2010-08-11,NaN,2,4,198230,1,1


In [13]:
df[df['istest']==1]['Emp_ID'].nunique()

741

In [14]:
df.describe(include=np.object)

,MMM-YY,Gender,City,Education_Level,Dateofjoining,LastWorkingDate
count,19104,19104,19104,19104,19104,1616
unique,24,2,29,3,869,493
top,2016-01-01,Male,C20,Bachelor,2012-07-23,2017-07-29
freq,1022,11103,1008,6864,192,70


In [15]:
df['joining_date'] = pd.to_datetime(df['Dateofjoining'])
df['reporting_date'] = pd.to_datetime(df['MMM-YY'])
df['lwd'] = pd.to_datetime(df['LastWorkingDate'])
df['city_code'] = df['City'].astype(str).str[1:]
df['male'] = np.where(df['Gender'] =='Male',1,0)
df['education_cat'] = df['Education_Level'].map({'Master':2,'College':0,'Bachelor':1})
df.sample(3)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,istest,joining_date,reporting_date,lwd,city_code,male,education_cat
7568,2016-05-01,1129,32,Male,C6,College,73087,2015-06-04,NaN,1,2,802600,4,0,2015-06-04,2016-05-01,NaT,6,1,0
9799,2016-05-01,1458,27,Female,C17,Master,41971,2015-09-01,NaN,2,2,0,1,0,2015-09-01,2016-05-01,NaT,17,0,2
6183,2016-07-01,926,47,Male,C4,College,21646,2013-01-07,NaN,1,2,317380,2,1,2013-01-07,2016-07-01,NaT,4,1,0


In [17]:
ref_date = datetime.datetime(2017, 12, 31)
df['tenure_days'] = (ref_date - df['joining_date']).dt.days
df.sample(3)                             

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,istest,joining_date,reporting_date,lwd,city_code,male,education_cat,tenure_days
1929,2017-12-01,306,31,Female,C4,College,53025,2017-11-29,NaN,2,2,0,1,1,2017-11-29,2017-12-01,NaT,4,0,0,32
16087,2017-01-01,2390,45,Male,C28,College,46244,2015-05-30,NaN,1,1,1377400,4,1,2015-05-30,2017-01-01,NaT,28,1,0,946
2300,2017-02-01,358,39,Female,C26,Master,117830,2016-10-29,NaN,3,3,300440,2,1,2016-10-29,2017-02-01,NaT,26,0,2,428


In [18]:
df['tenure_days'].describe()

count    19104.000000
mean       976.764604
std        693.335918
min          3.000000
25%        421.000000
50%        841.000000
75%       1492.500000
max       2831.000000
Name: tenure_days, dtype: float64

In [19]:
df['reporting_days'] = (df['reporting_date'] - df['joining_date']).dt.days
df['reporting_days'].describe()

count    19104.000000
mean       591.340557
std        651.901528
min        -27.000000
25%         87.000000
50%        297.000000
75%       1007.000000
max       2801.000000
Name: reporting_days, dtype: float64

In [20]:
df['target'] = np.where(df['istest']==1,np.nan, np.where(df['LastWorkingDate'].isna(),0,1))
df.sample(3)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,istest,joining_date,reporting_date,lwd,city_code,male,education_cat,tenure_days,reporting_days,target
15078,2016-04-01,2228,54,Female,C27,College,79243,2015-07-21,NaN,1,2,367320,3,1,2015-07-21,2016-04-01,NaT,27,0,0,894,255,NaN
6998,2017-01-01,1049,28,Male,C26,Master,37278,2015-05-09,NaN,1,1,954330,3,0,2015-05-09,2017-01-01,NaT,26,1,2,967,603,0.0
6332,2017-04-01,945,39,Female,C3,Master,83025,2012-05-29,NaN,2,3,350000,3,1,2012-05-29,2017-04-01,NaT,3,0,2,2042,1768,NaN


In [21]:
df['target'].value_counts(normalize=True)

0.0    0.846329
1.0    0.153671
Name: target, dtype: float64

In [22]:
df.groupby(['Quarterly Rating'])['target'].mean().mul(100)

Quarterly Rating
1    23.906511
2     5.299456
3     2.172227
4     2.074689
Name: target, dtype: float64

In [23]:
df.groupby(['target'])['reporting_days'].mean().mul(100)

target
0.0    43956.393258
1.0    34708.663366
Name: reporting_days, dtype: float64

In [24]:
df[df['istest']==1]['Emp_ID'].nunique(), df[df['istest']==0]['Emp_ID'].nunique(), df['Emp_ID'].nunique()

(741, 1640, 2381)

In [25]:
sal = df.groupby(['Emp_ID'])['Salary'].agg(['min','max']).add_prefix('salary_')
sal['sal_rev'] = sal['salary_max'] -sal['salary_min']
sal['per_change'] = sal['sal_rev'].div(sal['salary_min'])
sal['sal_change'] = (sal['salary_min'] != sal['salary_max']).astype(int)
sal.sample(3)

,salary_min,salary_max,sal_rev,per_change,sal_change
Emp_ID,,,,,
928,67236,67236,0,0.0,0
1836,48546,48546,0,0.0,0
1115,64444,64444,0,0.0,0


In [26]:
sal.describe()

,salary_min,salary_max,sal_rev,per_change,sal_change
count,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000
mean,59209.060899,59336.159597,127.098698,0.001839,0.018480
std,28275.899087,28383.012146,973.171018,0.013451,0.134706
min,10747.000000,10747.000000,0.000000,0.000000,0.000000
25%,39104.000000,39104.000000,0.000000,0.000000,0.000000
50%,55276.000000,55315.000000,0.000000,0.000000,0.000000
75%,75765.000000,75986.000000,0.000000,0.000000,0.000000
max,188418.000000,188418.000000,12155.000000,0.111139,1.000000


In [27]:
record_count = df['Emp_ID'].value_counts().to_frame()
record_count.columns = ['count']
record_count.head(3)

,count
2049,24
1191,24
1655,24


In [28]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [29]:
bus_val = df.groupby(['Emp_ID'])['Total Business Value'].agg(['min','max','mean','std',percentile(50),percentile(25),percentile(75)])
bus_val.head(3)

,min,max,mean,std,percentile_50,percentile_25,percentile_75
Emp_ID,,,,,,,
1,-665480,2381060,571860.0,1.601755e+06,0,-332740.0,1190530.0
2,0,0,0.0,0.000000e+00,0,0.0,0.0
4,0,350000,70000.0,1.565248e+05,0,0.0,0.0


In [30]:
sub = df[df['Emp_ID']==1]
sub = sub.sort_values(['reporting_date'],ascending=True)
sub.head(3)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,istest,joining_date,reporting_date,lwd,city_code,male,education_cat,tenure_days,reporting_days,target
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2,0,2015-12-24,2016-01-01,NaT,23,1,2,738,8,0.0
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2,0,2015-12-24,2016-02-01,NaT,23,1,2,738,39,0.0
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,0,2015-12-24,2016-03-01,2016-03-11,23,1,2,738,68,1.0


In [31]:
cols = ['mean','std','min','25%','50%','75%','max'] 
rol_bus_val = pd.DataFrame(columns=cols+['inc'])
for cnt,emp in enumerate(df['Emp_ID'].unique()):
    print(f"\rPrint processing {cnt+1}/{df['Emp_ID'].nunique()}...",end='')
    sub = df[df['Emp_ID']==emp]
    sub = sub.sort_values(by=['reporting_date'],ascending=True)
    rol_bus_val.loc[emp,cols] = sub['Total Business Value'].diff().describe()[cols].values
    rol_bus_val.at[emp,'inc'] = sub['Total Business Value'].values[-1] -sub['Total Business Value'].values[0] 
rol_bus_val.head(3)

Print processing 2381/2381...

,mean,std,min,25%,50%,75%,max,inc
1,-1.19053e+06,2.62479e+06,-3.04654e+06,-2.11854e+06,-1.19053e+06,-262525,665480,-2381060
2,0,NaN,0,0,0,0,0,0
4,0,285774,-350000,-87500,0,87500,350000,0


In [32]:
rol_bus_val = rol_bus_val.add_prefix("rolling_")
rol_bus_val.head(3)

,rolling_mean,rolling_std,rolling_min,rolling_25%,rolling_50%,rolling_75%,rolling_max,rolling_inc
1,-1.19053e+06,2.62479e+06,-3.04654e+06,-2.11854e+06,-1.19053e+06,-262525,665480,-2381060
2,0,NaN,0,0,0,0,0,0
4,0,285774,-350000,-87500,0,87500,350000,0


In [33]:
rol_bus_val = rol_bus_val.astype(float)
rol_bus_val.describe()

,rolling_mean,rolling_std,rolling_min,rolling_25%,rolling_50%,rolling_75%,rolling_max,rolling_inc
count,2.200000e+03,1.997000e+03,2.200000e+03,2.200000e+03,2.200000e+03,2.200000e+03,2.200000e+03,2.381000e+03
mean,-7.615599e+03,6.347031e+05,-1.131274e+06,-2.093298e+05,3.435782e+03,2.128743e+05,1.051141e+06,-2.440566e+04
std,1.619176e+05,8.625677e+05,1.993555e+06,3.189296e+05,1.737177e+05,3.377773e+05,2.063986e+06,1.182482e+06
min,-4.513790e+06,0.000000e+00,-1.740943e+07,-4.513790e+06,-4.513790e+06,-4.513790e+06,-4.513790e+06,-9.658160e+06
25%,-2.394333e+04,1.216639e+05,-1.261695e+06,-3.047512e+05,0.000000e+00,0.000000e+00,0.000000e+00,-1.166600e+05
50%,0.000000e+00,3.819441e+05,-4.450650e+05,-1.276488e+05,0.000000e+00,1.206512e+05,4.000000e+05,0.000000e+00
75%,0.000000e+00,8.087101e+05,0.000000e+00,0.000000e+00,1.084625e+04,3.307438e+05,1.156385e+06,0.000000e+00
max,1.365897e+06,9.375143e+06,7.467200e+05,7.467200e+05,1.050975e+06,2.828552e+06,3.505256e+07,2.391754e+07


In [34]:
rating_count = df.groupby(['Emp_ID','Quarterly Rating']).size().unstack(fill_value=0)
rating_count = rating_count.add_prefix('rating_')
rating_count.head(3)

Quarterly Rating,rating_1,rating_2,rating_3,rating_4
Emp_ID,,,,
1,0,3,0,0
2,2,0,0,0
4,5,0,0,0


In [35]:
rating_summ = df.groupby(['Emp_ID'])['Quarterly Rating'].agg(['min','max','mean'])
rating_summ = rating_summ.add_prefix('rating_')
rating_summ.head(3)

,rating_min,rating_max,rating_mean
Emp_ID,,,
1,2,2,2.0
2,1,1,1.0
4,1,1,1.0


In [36]:
cols = ['mean','std','min','75%','max'] 
rol_rating = pd.DataFrame(columns=cols+['inc'])
for cnt,emp in enumerate(df['Emp_ID'].unique()):
    print(f"\rPrint processing {cnt+1}/{df['Emp_ID'].nunique()}...",end='')
    sub = df[df['Emp_ID']==emp]
    sub = sub.sort_values(by=['reporting_date'],ascending=True)
    rol_rating.loc[emp,cols] = sub['Quarterly Rating'].diff().describe()[cols].values
    rol_rating.at[emp,'inc'] = sub['Quarterly Rating'].values[-1] -sub['Quarterly Rating'].values[0] 
rol_rating.head(3)

Print processing 2381/2381...

,mean,std,min,75%,max,inc
1,0,0,0,0,0,0
2,0,NaN,0,0,0,0
4,0,0,0,0,0,0


In [37]:
rol_rating = rol_rating.fillna(0).add_prefix("rol_rat_")
rol_rating.head(3)

,rol_rat_mean,rol_rat_std,rol_rat_min,rol_rat_75%,rol_rat_max,rol_rat_inc
1,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0


In [38]:
rol_rating.describe()

,rol_rat_mean,rol_rat_std,rol_rat_min,rol_rat_75%,rol_rat_max,rol_rat_inc
count,2381.000000,2381.000000,2381.00000,2381.000000,2381.000000,2381.000000
mean,-0.006138,0.283461,-0.52121,0.016800,0.478370,-0.058379
std,0.137995,0.354149,0.76317,0.099533,0.739673,0.933703
min,-1.000000,0.000000,-3.00000,-0.250000,0.000000,-3.000000
25%,0.000000,0.000000,-1.00000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,0.000000,0.500000,0.00000,0.000000,1.000000,0.000000
max,1.000000,1.732051,0.00000,1.500000,3.000000,3.000000


In [39]:
join_des = df.groupby(['Emp_ID','Joining Designation']).size().unstack(fill_value=0)
join_des = join_des.add_prefix("join_des_")
join_des.head(3)

Joining Designation,join_des_1,join_des_2,join_des_3,join_des_4,join_des_5
Emp_ID,,,,,
1,3,0,0,0,0
2,0,2,0,0,0
4,0,5,0,0,0


In [40]:
des = df.groupby(['Emp_ID','Designation']).size().unstack(fill_value=0)
des = des.add_prefix("des_")
des.head(3)

Designation,des_1,des_2,des_3,des_4,des_5
Emp_ID,,,,,
1,3,0,0,0,0
2,0,2,0,0,0
4,0,5,0,0,0


In [41]:
df['des_change'] = (df['Joining Designation'] != df['Designation']).astype(int)
des_change = df.groupby(['Emp_ID'])['des_change'].max().to_frame()
des_change.columns = ['des_change']
des_change.head(3)

,des_change
Emp_ID,
1,0
2,0
4,0


In [42]:
des_change['promote'] = 0
for cnt,emp in enumerate(df['Emp_ID'].unique()):
    print(f"\rPrint processing {cnt+1}/{df['Emp_ID'].nunique()}...",end='')
    sub = df[df['Emp_ID']==emp]
    sub = sub.sort_values(by=['reporting_date'],ascending=True)
    seq = sub['Designation'].diff().values.tolist()

    if 1.0 in seq:
        des_change.at[emp,'promote'] = seq.index(1.0)
des_change.head(3)

Print processing 2381/2381...

,des_change,promote
Emp_ID,,
1,0,0
2,0,0
4,0,0


In [43]:
des_change.describe()

,des_change,promote
count,2381.000000,2381.000000
mean,0.171357,0.343553
std,0.376899,2.547777
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,23.000000


In [44]:
age = df.groupby(['Emp_ID'])['Age'].agg(['min','max','mean'])
age['exp_yrs'] = age['max'] -age['min']
age = age.add_prefix('age_')
age.describe()

,age_min,age_max,age_mean,age_exp_yrs
count,2381.000000,2381.000000,2381.000000,2381.000000
mean,33.090298,33.663167,33.369050,0.572869
std,5.840729,5.983375,5.890619,0.691062
min,21.000000,21.000000,21.000000,0.000000
25%,29.000000,29.000000,29.000000,0.000000
50%,33.000000,33.000000,33.000000,0.000000
75%,37.000000,37.000000,37.000000,1.000000
max,58.000000,58.000000,58.000000,2.000000


In [45]:
df.head(3)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,istest,joining_date,reporting_date,lwd,city_code,male,education_cat,tenure_days,reporting_days,target,des_change
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2,0,2015-12-24,2016-01-01,NaT,23,1,2,738,8,0.0,0
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2,0,2015-12-24,2016-02-01,NaT,23,1,2,738,39,0.0,0
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2,0,2015-12-24,2016-03-01,2016-03-11,23,1,2,738,68,1.0,0


In [46]:
tenure = df.groupby(['Emp_ID'])[['tenure_days','reporting_days']].mean()
tenure.head(3)

,tenure_days,reporting_days
Emp_ID,,
1,738.0,38.333333
2,55.0,10.000000
4,389.0,54.800000


In [47]:
tenure['tenure_in_month']= tenure['tenure_days'].div(30)
tenure['reporting_in_month'] = tenure['reporting_days'].div(30)
tenure.sample(3)

,tenure_days,reporting_days,tenure_in_month,reporting_in_month
Emp_ID,,,,
1083,429.0,34.400000,14.300000,1.146667
614,802.0,224.090909,26.733333,7.469697
327,947.0,262.500000,31.566667,8.750000


In [48]:
demo = df.groupby(['Emp_ID'])[['male','education_cat','city_code']].max()
demo.head(3)

,male,education_cat,city_code
Emp_ID,,,
1,1,2,23
2,1,2,7
4,1,2,13


In [49]:
join_df = df.loc[df.groupby(['Emp_ID'])['reporting_date'].idxmin(),:]
join_perform = join_df.groupby(['Age','Gender','City','Education_Level'])[['Salary','Total Business Value','Designation','Quarterly Rating']].mean()
join_perform = join_perform.add_prefix("join_")
join_perform = join_perform.reset_index()
join_perform.head(3)

,Age,Gender,City,Education_Level,join_Salary,join_Total Business Value,join_Designation,join_Quarterly Rating
0,21,Female,C18,College,21673.0,0.0,1.0,1.0
1,21,Female,C6,College,22115.0,100010.0,1.0,1.0
2,21,Male,C10,College,25172.0,0.0,1.0,1.0


In [50]:
join_df=join_df.merge(join_perform,how='left',on=['Age','Gender','City','Education_Level'])
join_df.head(3)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,istest,joining_date,reporting_date,lwd,city_code,male,education_cat,tenure_days,reporting_days,target,des_change,join_Salary,join_Total Business Value,join_Designation,join_Quarterly Rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2,0,2015-12-24,2016-01-01,NaT,23,1,2,738,8,0.0,0,57387.0,2381060.0,1.0,2.0
1,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1,1,2017-11-06,2017-11-01,NaT,7,1,2,55,-5,NaN,0,70130.5,0.0,2.5,1.0
2,2016-12-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,0,1,0,2016-12-07,2016-12-01,NaT,13,1,2,389,-6,0.0,0,75042.0,118050.0,2.5,1.0


In [51]:
join_df['salary_flag'] = join_df['join_Salary'] < join_df['Salary']
join_df['business_flag'] = join_df['join_Total Business Value'] < join_df['Total Business Value']
join_df['des_flag'] = join_df['join_Designation'] < join_df['Designation']
join_df['rating_flag'] = join_df['join_Quarterly Rating'] < join_df['Quarterly Rating']
join_df.head(3)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating,istest,joining_date,reporting_date,lwd,city_code,male,education_cat,tenure_days,reporting_days,target,des_change,join_Salary,join_Total Business Value,join_Designation,join_Quarterly Rating,salary_flag,business_flag,des_flag,rating_flag
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2,0,2015-12-24,2016-01-01,NaT,23,1,2,738,8,0.0,0,57387.0,2381060.0,1.0,2.0,False,False,False,False
1,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1,1,2017-11-06,2017-11-01,NaT,7,1,2,55,-5,NaN,0,70130.5,0.0,2.5,1.0,False,False,False,False
2,2016-12-01,4,43,Male,C13,Master,65603,2016-12-07,NaN,2,2,0,1,0,2016-12-07,2016-12-01,NaT,13,1,2,389,-6,0.0,0,75042.0,118050.0,2.5,1.0,False,False,False,False


In [52]:
emp_flag = join_df[['Emp_ID','salary_flag','business_flag','des_flag','rating_flag']].set_index(['Emp_ID']).astype(int)
emp_flag.head(3)

,salary_flag,business_flag,des_flag,rating_flag
Emp_ID,,,,
1,0,0,0,0
2,0,0,0,0
4,0,0,0,0


In [53]:
emp_flag.describe()

,salary_flag,business_flag,des_flag,rating_flag
count,2381.000000,2381.000000,2381.000000,2381.000000
mean,0.228895,0.120118,0.161277,0.112558
std,0.420210,0.325167,0.367863,0.316118
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


In [54]:
df['report_mnt'] = df['reporting_date'].dt.month
df['report_qtr'] = df['reporting_date'].dt.quarter
df['report_yr'] = df['reporting_date'].dt.year

rpt_mnt = df.groupby(['Emp_ID','report_mnt']).size().unstack(fill_value=0).add_prefix("rpt_mnt_")
rpt_qtr = df.groupby(['Emp_ID','report_qtr']).size().unstack(fill_value=0).add_prefix("rpt_qtr_")
rpt_yr = df.groupby(['Emp_ID','report_yr']).size().unstack(fill_value=0).add_prefix("rpt_yr_")

In [55]:
join_date_df = df[['Emp_ID','joining_date']].drop_duplicates()
join_date_df['join_mnt'] = join_date_df['joining_date'].dt.month
join_date_df['join_qtr'] = join_date_df['joining_date'].dt.quarter
join_date_df['join_yr'] = join_date_df['joining_date'].dt.year
join_date_df = join_date_df.drop(['joining_date'],axis=1).set_index('Emp_ID').add_prefix("join_dt_")
join_date_df.head(3)

,join_dt_join_mnt,join_dt_join_qtr,join_dt_join_yr
Emp_ID,,,
1,12,4,2015
2,11,4,2017
4,12,4,2016


In [56]:
df['target'] = df['target'].fillna(0).astype(int)
target = df.groupby(['Emp_ID'])[['target','istest']].max()
target.head(3)

,target,istest
Emp_ID,,
1,1,0
2,0,1
4,1,0


In [57]:
target.shape, np.sum(target['istest']==1),np.sum(target['istest']==0)

((2381, 2), 741, 1640)

In [58]:
features_df = [
               sal,
               record_count,
               bus_val,
               rol_bus_val,
               rating_count,
               rating_summ,
               rol_rating,
               join_des,
               des,
               des_change,
               age,
               tenure,
               demo,
               emp_flag,
               rpt_mnt,
               rpt_qtr,
               rpt_yr,
               join_date_df,
               target
]

In [59]:
process_df = pd.concat(features_df,axis=1,ignore_index=False,sort=False)
process_df = process_df.fillna(0)
process_df.index.name = "Emp_ID"
process_df = process_df.reset_index()
process_df.sample(3)

,Emp_ID,salary_min,salary_max,sal_rev,per_change,sal_change,count,min,max,mean,std,percentile_50,percentile_25,percentile_75,rolling_mean,rolling_std,rolling_min,rolling_25%,rolling_50%,rolling_75%,rolling_max,rolling_inc,rating_1,rating_2,rating_3,rating_4,rating_min,rating_max,rating_mean,rol_rat_mean,rol_rat_std,rol_rat_min,rol_rat_75%,rol_rat_max,rol_rat_inc,join_des_1,join_des_2,join_des_3,join_des_4,join_des_5,des_1,des_2,des_3,des_4,des_5,des_change,promote,age_min,age_max,age_mean,age_exp_yrs,tenure_days,reporting_days,tenure_in_month,reporting_in_month,male,education_cat,city_code,salary_flag,business_flag,des_flag,rating_flag,rpt_mnt_1,rpt_mnt_2,rpt_mnt_3,rpt_mnt_4,rpt_mnt_5,rpt_mnt_6,rpt_mnt_7,rpt_mnt_8,rpt_mnt_9,rpt_mnt_10,rpt_mnt_11,rpt_mnt_12,rpt_qtr_1,rpt_qtr_2,rpt_qtr_3,rpt_qtr_4,rpt_yr_2016,rpt_yr_2017,join_dt_join_mnt,join_dt_join_qtr,join_dt_join_yr,target,istest
1678,1976,51181,51181,0,0.0,0,1,0,0,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,1,0,0,0,0,0,31,31,31.0,0,884.0,154.0,29.466667,5.133333,0,2,20,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,7,3,2015,1,0
566,655,40926,40926,0,0.0,0,5,0,201300,85886.0,86544.390228,101850,0.0,126280.0,0.0,118093.696981,-101850.0,-100050.0,-12215.0,87835.0,126280.0,0.0,5,0,0,0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0,0,5,0,0,0,0,5,0,0,0,0,0,35,35,35.0,0,726.0,56.6,24.200000,1.886667,1,0,13,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,3,2,0,0,5,0,1,1,2016,1,0
1821,2141,49533,49533,0,0.0,0,2,0,0,0.0,0.000000,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,2,0,0,0,0,2,0,0,0,0,28,28,28.0,0,186.0,18.5,6.200000,0.616667,1,2,16,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,2,0,0,2,6,2,2017,1,0


In [60]:
process_df.shape[0] == df['Emp_ID'].nunique()

True

In [61]:
process_df.shape, np.sum(process_df['istest']==1),np.sum(process_df['istest']==0),raw_test.shape

((2381, 85), 741, 1640, (741, 1))

In [62]:
process_df.describe()

,Emp_ID,salary_min,salary_max,sal_rev,per_change,sal_change,count,min,max,mean,std,percentile_50,percentile_25,percentile_75,rolling_mean,rolling_std,rolling_min,rolling_25%,rolling_50%,rolling_75%,rolling_max,rolling_inc,rating_1,rating_2,rating_3,rating_4,rating_min,rating_max,rating_mean,rol_rat_mean,rol_rat_std,rol_rat_min,rol_rat_75%,rol_rat_max,rol_rat_inc,join_des_1,join_des_2,join_des_3,join_des_4,join_des_5,des_1,des_2,des_3,des_4,des_5,des_change,promote,age_min,age_max,age_mean,age_exp_yrs,tenure_days,reporting_days,tenure_in_month,reporting_in_month,male,education_cat,salary_flag,business_flag,des_flag,rating_flag,rpt_mnt_1,rpt_mnt_2,rpt_mnt_3,rpt_mnt_4,rpt_mnt_5,rpt_mnt_6,rpt_mnt_7,rpt_mnt_8,rpt_mnt_9,rpt_mnt_10,rpt_mnt_11,rpt_mnt_12,rpt_qtr_1,rpt_qtr_2,rpt_qtr_3,rpt_qtr_4,rpt_yr_2016,rpt_yr_2017,join_dt_join_mnt,join_dt_join_qtr,join_dt_join_yr,target,istest
count,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.00000,2.381000e+03,2.381000e+03,2.381000e+03,2.381000e+03,2.381000e+03,2.381000e+03,2.381000e+03,2.381000e+03,2.381000e+03,2.381000e+03,2.381000e+03,2.381000e+03,2.381000e+03,2.381000e+03,2.381000e+03,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.00000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.00000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000,2381.000000
mean,1397.559009,59209.060899,59336.159597,127.098698,0.001839,0.018480,8.02352,-1.855245e+04,1.251616e+06,3.120854e+05,3.800797e+05,2.006572e+05,8.328189e+04,3.967169e+05,-7.036673e+03,5.323402e+05,-1.045276e+06,-1.934169e+05,3.174599e+03,1.966919e+05,9.712349e+05,-2.440566e+04,3.225115,2.332213,1.635867,0.830323,1.143217,1.929861,1.566304,-0.006138,0.283461,-0.52121,0.016800,0.478370,-0.058379,4.128937,2.501050,1.195716,0.143217,0.054599,2.184796,2.783284,2.026879,0.900462,0.128097,0.171357,0.343553,33.090298,33.663167,33.369050,0.572869,691.379672,310.305444,23.045989,10.343515,0.589668,1.00756,0.228895,0.120118,0.161277,0.112558,0.757665,0.716086,0.667367,0.639647,0.642587,0.628307,0.656447,0.657707,0.659807,0.653927,0.666107,0.677866,2.141117,1.910542,1.973961,1.997900,4.077278,3.946241,7.357413,2.801764,2015.536329,0.678706,0.311214
std,806.161628,28275.899087,28383.012146,973.171018,0.013451,0.134706,6.78359,2.412859e+05,2.281539e+06,4.495705e+05,5.893408e+05,3.152723e+05,1.696131e+05,5.591779e+05,1.556520e+05,8.237074e+05,1.939574e+06,3.115440e+05,1.669839e+05,3.295466e+05,2.003423e+06,1.182482e+06,2.459874,3.716544,3.523111,2.591636,0.407989,1.104857,0.719652,0.137995,0.354149,0.76317,0.099533,0.739673,0.933703,6.978461,5.051884,2.975728,1.327182,0.903983,4.631763,5.295240,4.705096,4.054741,1.528157,0.376899,2.547777,5.840729,5.983375,5.890619,0.691062,586.993998,489.742624,19.566467,16.324754,0.491997,0.81629,0.420210,0.325167,0.367863,0.316118,0.698490,0.699962,0.698304,0.701983,0.697186,0.702353,0.703226,0.697544,0.690403,0.694769,0.697401,0.698114,2.018672,2.001884,1.982951,1.977073,4.408156,4.227318,3.143143,1.011786,1.609597,0.467071,0.463087
min,1.000000,10747.000000,10747.000000,0.000000,0.000000,0.000000,1.00000,-6.000000e+06,-1.011800e+05,-1.979329e+05,0.000000e+00,-1.011800e+05,-3.327400e+05,-1.011800e+05,-4.513790e+06,0.000000e+00,-1.740943e+07,-4.513790e+06,-4.513790e+06,-4.513790e+06,-4.513790e+06,-9.658160e+06,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,-1.000000,0.000000,-3.00000,-0.250000,0.000000,-3.000000,0

In [63]:
process_df.shape

(2381, 85)

In [85]:
process_df[['Emp_ID','target']].sample(5)

,Emp_ID,target
1893,2221,1
2043,2399,0
2033,2388,1
2340,2738,1
1898,2227,1


In [64]:
train = process_df[process_df['istest']==0].copy()
train.shape

(1640, 85)

In [65]:
# n_samples = 10000
# train_1 = resample(train[train['target']==1],replace=True,random_state=1423,n_samples=n_samples)
# train_1.shape

# train_2 = resample(train[train['target']==0],replace=True,random_state=123,n_samples=n_samples*3)
# train_2.shape

# train = pd.concat([train_1,train_2],axis=0,ignore_index=True,sort=False)
# train.shape

In [66]:
train['target'].value_counts(normalize=True)

1    0.985366
0    0.014634
Name: target, dtype: float64

In [67]:
x_train = train.drop(['Emp_ID','istest','target'],axis=1).astype(float)
x_train.head(3)

,salary_min,salary_max,sal_rev,per_change,sal_change,count,min,max,mean,std,percentile_50,percentile_25,percentile_75,rolling_mean,rolling_std,rolling_min,rolling_25%,rolling_50%,rolling_75%,rolling_max,rolling_inc,rating_1,rating_2,rating_3,rating_4,rating_min,rating_max,rating_mean,rol_rat_mean,rol_rat_std,rol_rat_min,rol_rat_75%,rol_rat_max,rol_rat_inc,join_des_1,join_des_2,join_des_3,join_des_4,join_des_5,des_1,des_2,des_3,des_4,des_5,des_change,promote,age_min,age_max,age_mean,age_exp_yrs,tenure_days,reporting_days,tenure_in_month,reporting_in_month,male,education_cat,city_code,salary_flag,business_flag,des_flag,rating_flag,rpt_mnt_1,rpt_mnt_2,rpt_mnt_3,rpt_mnt_4,rpt_mnt_5,rpt_mnt_6,rpt_mnt_7,rpt_mnt_8,rpt_mnt_9,rpt_mnt_10,rpt_mnt_11,rpt_mnt_12,rpt_qtr_1,rpt_qtr_2,rpt_qtr_3,rpt_qtr_4,rpt_yr_2016,rpt_yr_2017,join_dt_join_mnt,join_dt_join_qtr,join_dt_join_yr
0,57387.0,57387.0,0.0,0.0,0.0,3.0,-665480.0,2381060.0,571860.0,1.601755e+06,0.0,-332740.0,1190530.0,-1190530.0,2.624795e+06,-3046540.0,-2118535.0,-1190530.0,-262525.0,665480.0,-2381060.0,0.0,3.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,28.0,28.0,0.0,738.0,38.333333,24.600000,1.277778,1.0,2.0,23.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,12.0,4.0,2015.0
2,65603.0,65603.0,0.0,0.0,0.0,5.0,0.0,350000.0,70000.0,1.565248e+05,0.0,0.0,0.0,0.0,2.857738e+05,-350000.0,-87500.0,0.0,87500.0,350000.0,0.0,5.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,43.0,43.0,43.0,0.0,389.0,54.800000,12.966667,1.826667,1.0,2.0,13.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,1.0,1.0,4.0,12.0,4.0,2016.0
3,46368.0,46368.0,0.0,0.0,0.0,3.0,0.0,120360.0,40120.0,6.948988e+04,0.0,0.0,60180.0,0.0,1.702147e+05,-120360.0,-60180.0,0.0,60180.0,120360.0,0.0,3.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,29.0,29.0,0.0,722.0,22.333333,24.066667,0.744444,1.0,0.0,9.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,1.0,1.0,2016.0


In [68]:
y_train = train['target']
y_train.value_counts()

1    1616
0      24
Name: target, dtype: int64

In [69]:
scorer_func = make_scorer(f1_score)

In [86]:
rf = RandomForestClassifier(n_estimators=5,max_depth=3,class_weight='balanced')
rf_cv = cross_val_score(rf, x_train, y_train, cv=5,n_jobs=-1,scoring=scorer_func)
rf_cv

array([0.9504    , 0.909699  , 0.93831169, 0.9470305 , 0.92434211])

In [87]:
lg =  LogisticRegression(random_state=0,max_iter=1000,class_weight='balanced')
lg_piple = make_pipeline(StandardScaler(),lg)
lg_cv = cross_val_score(lg_piple, x_train, y_train, cv=5,n_jobs=-1,scoring=scorer_func)
lg_cv

array([0.94516129, 0.95008052, 0.94174757, 0.9398374 , 0.93506494])

In [91]:
xgb_clf = xgb.XGBClassifier(n_estimators=3,
                            max_depth =5,
                            learning_rate=0.001,
                            n_jobs=-1,
                            subsample=0.8,
                            colsample_bytree=0.8,
                            reg_alpha=0.0001,
                            reg_lambda=0.0001,
                            random_state=45)

xgb_cv = cross_val_score(xgb_clf, x_train, y_train, cv=5,n_jobs=-1,scoring=scorer_func)
xgb_cv

array([0.99386503, 0.99231951, 0.99231951, 0.99231951, 0.99076923])

In [92]:
lgb_clf = lgb.LGBMClassifier(learning_rate=0.001,
                             n_estimators=3,
                              subsample=0.8,
                             objective ='binary',
                             seed=45,
                             n_jobs=-1,
                             class_weight='balanced')
lgb_cv = cross_val_score(lgb_clf, x_train, y_train, cv=5,n_jobs=-1,scoring=scorer_func)
lgb_cv

array([0.93311582, 0.93658537, 0.92962357, 0.93851133, 0.94668821])

In [102]:
clf = lg_piple.fit(x_train,y_train)
predict = clf.predict(x_train)
print(classification_report(y_train,predict))

              precision    recall  f1-score   support

           0       0.13      1.00      0.22        24
           1       1.00      0.90      0.95      1616

    accuracy                           0.90      1640
   macro avg       0.56      0.95      0.59      1640
weighted avg       0.99      0.90      0.94      1640



In [108]:
ml_feature = x_train.columns
test = process_df[process_df['istest']==1].copy()
test['Target']= clf.predict(test[ml_feature].astype(float))
test = test[['Emp_ID','Target']].copy()
test['Target'] = np.where(test['Target']==1,0,1)
test.to_csv("Submission_3_LG.csv",index=False)